# Q-5. 
```
Uber is a taxi service provider as we know, we need to predict the high
booking area using an Unsupervised algorithm and price for the location using a
supervised algorithm and use some map function to display the data
Dataset This is the Dataset You can use this dataset for this question.
```

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("rideshare_kaggle.csv")

In [3]:
df.head()

,id,timestamp,hour,day,month,datetime,timezone,source,destination,cab_type,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,1.544953e+09,9,16,12,2018-12-16 09:30:07,America/New_York,Haymarket Square,North Station,Lyft,...,0.1276,1544979600,39.89,1545012000,43.68,1544968800,33.73,1545012000,38.07,1544958000
1,4bd23055-6827-41c6-b23b-3c491f24e74d,1.543284e+09,2,27,11,2018-11-27 02:00:23,America/New_York,Haymarket Square,North Station,Lyft,...,0.1300,1543251600,40.49,1543233600,47.30,1543251600,36.20,1543291200,43.92,1543251600
2,981a3613-77af-4620-a42a-0c0866077d1e,1.543367e+09,1,28,11,2018-11-28 01:00:22,America/New_York,Haymarket Square,North Station,Lyft,...,0.1064,1543338000,35.36,1543377600,47.55,1543320000,31.04,1543377600,44.12,1543320000
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,1.543554e+09,4,30,11,2018-11-30 04:53:02,America/New_York,Haymarket Square,North Station,Lyft,...,0.0000,1543507200,34.67,1543550400,45.03,1543510800,30.30,1543550400,38.53,1543510800
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,1.543463e+09,3,29,11,2018-11-29 03:49:20,America/New_York,Haymarket Square,North Station,Lyft,...,0.0001,1543420800,33.10,1543402800,42.18,1543420800,29.11,1543392000,35.75,1543420800


In [4]:
df.shape

(693071, 57)

In [5]:
df.isna().sum()

id                                 0
timestamp                          0
hour                               0
day                                0
month                              0
datetime                           0
timezone                           0
source                             0
destination                        0
cab_type                           0
product_id                         0
name                               0
price                          55095
distance                           0
surge_multiplier                   0
latitude                           0
longitude                          0
temperature                        0
apparentTemperature                0
short_summary                      0
long_summary                       0
precipIntensity                    0
precipProbability                  0
humidity                           0
windSpeed                          0
windGust                           0
windGustTime                       0
v

In [6]:
df_test = df[(df['price'].isnull())]
df = df[~(df['price'].isnull())]

In [7]:
df.shape

(637976, 57)

In [8]:
df.isna().sum()

id                             0
timestamp                      0
hour                           0
day                            0
month                          0
datetime                       0
timezone                       0
source                         0
destination                    0
cab_type                       0
product_id                     0
name                           0
price                          0
distance                       0
surge_multiplier               0
latitude                       0
longitude                      0
temperature                    0
apparentTemperature            0
short_summary                  0
long_summary                   0
precipIntensity                0
precipProbability              0
humidity                       0
windSpeed                      0
windGust                       0
windGustTime                   0
visibility                     0
temperatureHigh                0
temperatureHighTime            0
temperatur

In [9]:
num_cols = ['timestamp', 'distance',
       'surge_multiplier', 'latitude', 'longitude', 'temperature',
       'apparentTemperature', 'precipIntensity', 'precipProbability',
       'humidity', 'windSpeed', 'windGust', 'visibility',
       'temperatureHigh', 'temperatureLow', 'apparentTemperatureHigh','apparentTemperatureLow', 'dewPoint', 'pressure', 'windBearing',
       'cloudCover', 'uvIndex', 'visibility.1', 'ozone', 'moonPhase', 'precipIntensityMax',
       'temperatureMin', 'temperatureMax', 'apparentTemperatureMin', 'apparentTemperatureMax']

In [10]:
cat_cols = ['timezone', 'source', 'destination', 'cab_type',
       'product_id', 'name', 'short_summary', 'long_summary', 'icon']

In [11]:
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, LabelEncoder

In [12]:
cat_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder())
])

num_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

target_pipeline = Pipeline(steps=[
    ("encode", OrdinalEncoder())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, cat_cols),
    ("num", num_pipeline, num_cols)
])

In [13]:
from sklearn.cluster import KMeans
from kneed import KneeLocator
from sklearn.metrics import silhouette_score

In [14]:
from sklearn.model_selection import train_test_split

X = df.drop(["price"], axis=1)
y = df["price"]

In [15]:
train_arr = preprocessor.fit_transform(X)

In [40]:
from sklearn.cluster import DBSCAN

# Apply DBSCAN
dbscan = DBSCAN(eps=1, min_samples=100)
dbscan.fit(train_arr)

# Get the cluster labels
labels = dbscan.labels_

In [41]:
list(labels).count(-1)

253183

In [42]:
df.shape

(637976, 57)

In [55]:
df_final = pd.DataFrame(train_arr)

In [56]:
df_final["cluster"] = labels

In [57]:
df_final = pd.concat([df_final, y.reset_index(drop=True)], axis=1)

In [58]:
df_final["cluster"].value_counts()

-1       253183
 270        578
 396        573
 205        435
 101        432
          ...  
 2543         1
 2542         1
 2512         1
 2478         1
 2602         1
Name: cluster, Length: 2604, dtype: int64

In [59]:
df_final = df_final[~(df_final["cluster"]==-1)]

In [60]:
df_most = df_final[df_final["cluster"]==270]

In [61]:
df_most.shape

(578, 41)

In [62]:
df_most.isna().sum()

0          0
1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
10         0
11         0
12         0
13         0
14         0
15         0
16         0
17         0
18         0
19         0
20         0
21         0
22         0
23         0
24         0
25         0
26         0
27         0
28         0
29         0
30         0
31         0
32         0
33         0
34         0
35         0
36         0
37         0
38         0
cluster    0
price      0
dtype: int64

In [64]:
df_most.columns = cat_cols+num_cols+["cluster", "price"]

In [65]:
X = df_most.drop(["price"], axis=1)
y = df_most["price"]

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [67]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404 entries, 457028 to 98849
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timezone                 404 non-null    float64
 1   source                   404 non-null    float64
 2   destination              404 non-null    float64
 3   cab_type                 404 non-null    float64
 4   product_id               404 non-null    float64
 5   name                     404 non-null    float64
 6   short_summary            404 non-null    float64
 7   long_summary             404 non-null    float64
 8   icon                     404 non-null    float64
 9   timestamp                404 non-null    float64
 10  distance                 404 non-null    float64
 11  surge_multiplier         404 non-null    float64
 12  latitude                 404 non-null    float64
 13  longitude                404 non-null    float64
 14  temperature        

In [68]:
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error

# Create objects for the regression algorithms
models = {
    "linear_regression": LinearRegression(),
    "ridge_regression": Ridge(random_state=42),
    "lasso": Lasso(random_state=42),
    "extra_trees": ExtraTreesRegressor(random_state=42),
    "catboost": CatBoostRegressor(random_state=42, verbose=0),
    "knn": KNeighborsRegressor(),
    "lgbm": LGBMRegressor(random_state=42),
    "adaboost": AdaBoostRegressor(random_state=42),
    "elastic_net": ElasticNet(random_state=42),
    "random_forest": RandomForestRegressor(random_state=42),
    "decision_tree": DecisionTreeRegressor(random_state=42),
    "gradient_boosting": GradientBoostingRegressor(random_state=42),
}

In [71]:
r2_accuracy = []
trained_models_list = []
rmse_list = []

for model in list(models.values()):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    score = r2_score(y_test, y_pred)
    r2_accuracy.append(score)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_list.append((rmse))
    trained_models_list.append(model)

In [72]:
r2_max_value = max(r2_accuracy)
r2_max_index = r2_accuracy.index(r2_max_value)
best_model = trained_models_list[r2_max_index]
best_model_name = list(models.keys())[r2_max_index]

In [73]:
print(f"{best_model_name}: {round(r2_max_value, 2)*100}")

gradient_boosting: 94.0
